<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>
<h2 align=center><font size = 5>Part 2: Neighborhoods in Toronto City with geographical coordinates </font></h2>


## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in Toronto City. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in Toronto City and their emerging clusters.

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print('Libraries imported.')


Libraries imported.


### Read CSV genersted in part 1

In [37]:
toronto_Neighborhoods_pd = pd.read_csv('Toronto_Neighborhoods.csv')
toronto_Neighborhoods_pd.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


<a id='item1'></a>

### Try to get the latitude and the longitude coordinates of each neighborhood.
### 1- using geocoder package input:postal_code

In [ ]:
#install conda install -c conda-forge geocoder
#!conda install geocoder
#!pip install git+https://github.com/geopandas/geopandas
import geocoder # import geocoder
def get_lat_lng_coords(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    counter =0
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        counter +=1
        if counter>6: 
            return None
            
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

print(get_lat_lng_coords( 'M5G'  ) )

#### unreliable package and didn't provide location 

### 2- using geopy.geocoders package  input : address 

In [26]:
# define function to get location of address 
def get_geo_location(address):
    geolocator = Nominatim(user_agent="tor_explorer")
    location = geolocator.geocode(  address ,exactly_one=True )

    if location==None:
        print(address, 'Return None')
        return None
    print(address, location.latitude,location.longitude)

    return location.latitude,location.longitude 

get_geo_location('Harbourfront, Downtown Toronto, Toronto') 
get_geo_location('Parkwoods, North York, Toronto') 
print()

Harbourfront, Downtown Toronto, Toronto Return None
Parkwoods, North York, Toronto 43.7587999 -79.3201966



#### as shown above sample from address to get location, some address return Nonde as shown  

In [36]:
#get location of all Neighborhood
#toronto_Neighborhoods_pd['location'] =  '{postalcode:'+ toronto_Neighborhoods_pd['Postcode'] +'}'
#toronto_Neighborhoods_pd['location'] =  toronto_Neighborhoods_pd['Neighborhood'] +', '+  toronto_Neighborhoods_pd[  'Borough'] + ', Toronto' 
#toronto_Neighborhoods_pd['location'] = toronto_Neighborhoods_pd['location'].apply(get_geo_location)

#neighborhoods.head()

### Read CSV of geographical coordinates of each postal code from http://cocl.us/Geospatial_data  

In [39]:
# read CSV genersted in part 1
geos_coordinates_pd = pd.read_csv('Geospatial_Coordinates.csv')
geos_coordinates_pd.head(3)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


### Merge geospatial_coordinates_pd with toronto_Neighborhoods_pd to add latitude/longitude for each neighborhood


In [41]:
neighborhoods= toronto_Neighborhoods_pd.copy()
neighborhoods = neighborhoods.join(geos_coordinates_pd.set_index('Postal Code'), on='Postcode')

In [42]:
neighborhoods.head(12)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [44]:
neighborhoods.to_csv('Merge_Toronto_Neighborhoods_with_coordinates.csv', index=False)